In [1]:
%pip install pandas
%pip install numpy
%pip install faker

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# List of Malaysian states
malaysian_states = [
    'Johor', 'Kedah', 'Kelantan', 'Kuala Lumpur', 'Labuan', 'Malacca', 
    'Negeri Sembilan', 'Pahang', 'Penang', 'Perak', 'Perlis', 'Putrajaya', 
    'Sabah', 'Sarawak', 'Selangor', 'Terengganu'
]

# Set random seed for reproducibility
np.random.seed(42)

# Generate a realistic age distribution (e.g., a bell curve centered around 35)
ages = np.random.normal(loc=35, scale=10, size=30000).astype(int)
ages = np.clip(ages, 18, 70)  # Ensure ages are between 18 and 70

# Generate registration dates over the past 8 years
end_date = datetime.now()
start_date = end_date - timedelta(days=8*365)
registration_dates = [fake.date_between(start_date=start_date, end_date=end_date - timedelta(days=30)) for _ in range(30000)]

# Generate last login dates, with some users not having logged in recently to simulate churn
last_login_dates = []
for reg_date in registration_dates:
    if np.random.rand() < 0.3:  # 30% chance the user hasn't logged in the past 30 days
        last_login_date = fake.date_between(start_date=reg_date, end_date=end_date - timedelta(days=30))
    else:  # 70% chance the user has logged in recently (within the last 30 days)
        last_login_date = fake.date_between(start_date=end_date - timedelta(days=30), end_date=end_date)
    last_login_dates.append(last_login_date)

# Generate unique User data
users_data = []
usernames = set()
while len(users_data) < 30000:
    username = fake.user_name()
    if username not in usernames:
        usernames.add(username)
        age = ages[len(users_data)]
        state = np.random.choice(malaysian_states)
        registration_date = registration_dates[len(users_data)]
        last_login_date = last_login_dates[len(users_data)]
        users_data.append([username, age, state, registration_date, last_login_date])

# Create Users DataFrame
users_columns = ['Username', 'Age', 'State', 'Registration Date', 'Last Login Date']
users_df = pd.DataFrame(users_data, columns=users_columns)

# Generate Sessions data
n_sessions = 100000  # Assuming each user has multiple sessions
sessions_data = []

for _ in range(n_sessions):
    session_id = fake.uuid4()
    user = users_df.sample(1).iloc[0]
    username = user['Username']
    reg_date = user['Registration Date']
    last_login_date = user['Last Login Date']
    session_date = fake.date_between(start_date=reg_date, end_date=last_login_date)
    number_of_logins = max(np.random.poisson(lam=50), 1)  # Ensure at least 1 login
    transaction_volume = np.random.exponential(scale=1500)
    # Round transaction_volume to two decimal places
    transaction_volume = round(transaction_volume, 2)
    
    sessions_data.append([session_id, username, session_date, number_of_logins, transaction_volume])

# Create Sessions DataFrame
sessions_columns = ['Session ID', 'Username', 'Session Date', 'Number of Logins', 'Transaction Volume']
sessions_df = pd.DataFrame(sessions_data, columns=sessions_columns)

# Save to CSV files (with error handling)
try:
    users_df.to_csv('users.csv', index=False)
    sessions_df.to_csv('sessions.csv', index=False)
    print("Data saved successfully.")
except Exception as e:
    print(f"Error saving data: {e}")

Data saved successfully.


In [3]:
users_df.head()

,Username,Age,State,Registration Date,Last Login Date
0,yjohnson,39,Putrajaya,2022-12-19,2024-06-18
1,therrera,33,Perak,2018-12-01,2024-06-16
2,christopher50,41,Pahang,2021-03-05,2024-06-06
3,smithsara,50,Terengganu,2016-10-22,2024-06-21
4,joshua33,32,Labuan,2020-05-09,2024-06-12


In [4]:
users_df.tail()

,Username,Age,State,Registration Date,Last Login Date
29995,romanpamela,29,Penang,2018-10-04,2020-01-01
29996,butlerpatricia,49,Putrajaya,2019-11-08,2023-12-01
29997,wendyryan,29,Sabah,2017-03-17,2024-06-28
29998,shariweaver,18,Pahang,2023-10-01,2024-06-14
29999,mmcdaniel,32,Negeri Sembilan,2023-03-06,2024-05-30


In [5]:
sessions_df.head()

,Session ID,Username,Session Date,Number of Logins,Transaction Volume
0,b33a6ddb-efa4-48b6-b38c-a876f00729fa,erin38,2019-02-03,50,2841.35
1,bd02e116-6e63-47d5-84ea-88b54cc969a1,price,2023-11-29,41,28.85
2,2e2736f4-6a91-412b-9f67-dad2f865bffd,bartlettmichael,2022-05-10,45,869.63
3,be84fb62-fffc-4bf2-9afb-9fca873f54ec,xcarpenter,2021-08-22,52,6053.94
4,08a81d34-a368-49c7-bb43-a1e9f4391431,jeffreygibson,2017-08-18,61,570.83


In [6]:
sessions_df.tail()

,Session ID,Username,Session Date,Number of Logins,Transaction Volume
99995,e8e1a527-bca2-4127-8b63-7f84fbc11e43,monroeheather,2024-02-08,50,398.57
99996,1d490f93-b762-4413-a7ef-6b59d99d2265,bakerjennifer,2022-11-13,49,259.72
99997,540e067f-c2c6-4a11-b9f8-99ffa24b216f,aaronmora,2022-11-02,55,1850.40
99998,20e07f6d-d6ed-41d1-8e3f-a30afa73203d,hannah09,2022-06-23,63,330.44
99999,66b3c909-a345-40ed-b14c-122e0e4490af,kfranklin,2023-03-16,48,2094.37
